# 비상장 중소기업의 상장 후 주가 예측 프로젝트

이 프로젝트는 비상장 중소기업의 상장 후 주가 예측 프로젝트다. 과거 중소기업이 상장 후 주가의 산술평균을 타겟값으로 지도 학습 알고리즘을 사용할 것이다.

기본설정

In [1]:
import numpy as np 
import os  
import matplotlib ##그림용
import matplotlib.pyplot as plt ##그림용
import sklearn ##사이킷런
import pandas as pd

np.random.seed(42) ##랜덤시드 

matplotlib.rc('font', family='NanumBarunGothic') ##matplotlib 한글을 위해
plt.rcParams['axes.unicode_minus'] = False ##한글을 위해

In [2]:
data = pd.read_csv('datasets//DB//DB_data.csv', engine='python')

## 기업규모명

우리가 예측하고 싶은 대상은 중소기업이므로 중소기업만 관찰한다.

In [3]:
data = data[data["기업규모명"] == '중소기업']

데이터를 간략하게 살펴보자

In [4]:
data.head()

,회사명,회계년도,산업명,소속코드,설립일,종업원,당좌자산(*),재고자산(*),비유동자산(*),유동부채(*),...,환율변동으로 인한 차이조정,현금의 증가(감소)(*),기업규모명,산술평균,경제성장률,총저축률,제조업 평균가동률,실업률,코스피지수,경상수지
0,(주)CMG제약,2011-03-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,9917659.0,3104211.0,15477735.0,4969819.0,...,0.0,-265171.0,중소기업,873.0,1.0,34.1,81.3,4.2,1939.30,-4253.0
1,(주)CMG제약,2011-06-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,11341653.0,3071427.0,14731279.0,5976935.0,...,0.0,-1160724.0,중소기업,577.0,0.6,34.1,80.6,3.4,2142.47,-2470.6
2,(주)CMG제약,2011-09-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,11407699.0,3352328.0,14717723.0,6389723.0,...,0.0,-1961689.0,중소기업,993.0,0.7,34.8,80.9,3.1,1880.11,11405.6
3,(주)CMG제약,2011-12-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,12521782.0,2743103.0,12283654.0,7620349.0,...,-14.0,-1294857.0,중소기업,775.0,0.6,35.2,79.3,2.9,1847.51,11956.2
4,(주)CMG제약,2012-03-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,13370740.0,3654331.0,12100895.0,8146944.0,...,-306.0,435947.0,중소기업,892.0,0.5,34.3,79.7,3.8,2045.08,-1292.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21047 entries, 0 to 47822
Data columns (total 40 columns):
회사명                  21047 non-null object
회계년도                 21047 non-null object
산업명                  20908 non-null object
소속코드                 20908 non-null float64
설립일                  20908 non-null object
종업원                  20908 non-null float64
당좌자산(*)              20615 non-null float64
재고자산(*)              20615 non-null float64
비유동자산(*)             20615 non-null float64
유동부채(*)              20615 non-null float64
비유동부채(*)             20615 non-null float64
자본금(*)               20615 non-null float64
자본잉여금(*)             20615 non-null float64
자본조정(*)              20615 non-null float64
기타포괄손익누계액(*)         20615 non-null float64
이익잉여금(*)             20615 non-null float64
매출액(영업수익)(*)         20615 non-null float64
매출원가(*)              20615 non-null float64
매출총이익(손실)            20615 non-null float64
판매비와관리비(영업비용)(*)     20615 non-null float64
영업이익(손실)       

## 파일 정제

결측치가 몇 개 보인다.  산술평균에 경우 우리의 타겟값이므로 산술평균의 결측치가 있는 경우 객체를 버린다.

In [6]:
data = data[-(data["산술평균"].isnull())]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19416 entries, 0 to 47822
Data columns (total 40 columns):
회사명                  19416 non-null object
회계년도                 19416 non-null object
산업명                  19416 non-null object
소속코드                 19416 non-null float64
설립일                  19416 non-null object
종업원                  19416 non-null float64
당좌자산(*)              19145 non-null float64
재고자산(*)              19145 non-null float64
비유동자산(*)             19145 non-null float64
유동부채(*)              19145 non-null float64
비유동부채(*)             19145 non-null float64
자본금(*)               19145 non-null float64
자본잉여금(*)             19145 non-null float64
자본조정(*)              19145 non-null float64
기타포괄손익누계액(*)         19145 non-null float64
이익잉여금(*)             19145 non-null float64
매출액(영업수익)(*)         19145 non-null float64
매출원가(*)              19145 non-null float64
매출총이익(손실)            19145 non-null float64
판매비와관리비(영업비용)(*)     19145 non-null float64
영업이익(손실)       

이 경우 결측치가 있는 데이터가 소수일 것이라 생각되어 결측치가 있는 데이터를 버리는 것이 좋을 것 같다.

In [7]:
data = data[-data["당좌자산(*)"].isnull()]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19145 entries, 0 to 47822
Data columns (total 40 columns):
회사명                  19145 non-null object
회계년도                 19145 non-null object
산업명                  19145 non-null object
소속코드                 19145 non-null float64
설립일                  19145 non-null object
종업원                  19145 non-null float64
당좌자산(*)              19145 non-null float64
재고자산(*)              19145 non-null float64
비유동자산(*)             19145 non-null float64
유동부채(*)              19145 non-null float64
비유동부채(*)             19145 non-null float64
자본금(*)               19145 non-null float64
자본잉여금(*)             19145 non-null float64
자본조정(*)              19145 non-null float64
기타포괄손익누계액(*)         19145 non-null float64
이익잉여금(*)             19145 non-null float64
매출액(영업수익)(*)         19145 non-null float64
매출원가(*)              19145 non-null float64
매출총이익(손실)            19145 non-null float64
판매비와관리비(영업비용)(*)     19145 non-null float64
영업이익(손실)       

결측치가 있는 데이터가 없다.

## 머신러닝에 넣기 위해 데이터를 전처리하자.

train 데이터와 test 데이터를 나누자.

In [8]:
from sklearn.model_selection import train_test_split

train,test = train_test_split(data, test_size=0.2, random_state=42)

test의 경우 나중에 모델을 평가할 때 사용하자.

In [9]:
labels = train["산술평균"]
train.drop(["산술평균"], axis=1, inplace=True)

c:\users\student\documents\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


주어진 미시 변수에 경우 서로의 상관관계가 높을 것이라 생각되어, PCA알고리즘을 이용하여 차원을 줄이도록 하자.

In [13]:
micro_val = ['당좌자산(*)','재고자산(*)','비유동자산(*)','유동부채(*)','비유동부채(*)',
        '자본금(*)','자본잉여금(*)','자본조정(*)','기타포괄손익누계액(*)',
        '이익잉여금(*)','매출액(영업수익)(*)','매출원가(*)','매출총이익(손실)',
        '판매비와관리비(영업비용)(*)','영업이익(손실)','영업외수익(*)','영업외비용(*)',
        '법인세비용차감전(계속사업)손익','(계속사업손익)법인세비용','계속사업이익(손실)',
        '당기순이익(순손실)','영업활동으로 인한 현금흐름(*)',
        '투자활동으로 인한 현금흐름(*)','재무활동으로 인한 현금흐름(*)',
       '환율변동으로 인한 차이조정','현금의 증가(감소)(*)']

X = train[micro_val]

먼저 주어진 데이터를 95%이상 설명가능한 차원의 수를 구하자.

In [14]:
from sklearn.decomposition import PCA

In [16]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_) ## 모든 배열의 원소들을 누적으로 다 더한 놈들 반환
d = np.argmax(cumsum>=0.95)+1 ## argmax는 제일 큰 수의 index를 1차원 배열로 봤을 때의 index로 반환
# d = 9
## 9차원이면 주어진 데이터를 95%이상 설명할 수 있다.
pca = PCA(n_components=d)

pca.fit_transform(X)

array([[-48247380.59715018,  -4110257.20172153,  20321826.26031258, ...,
           816100.10330276,   -703788.93327897,  -6149054.05429247],
       [-65519713.17816956,  36031202.3647343 ,  10882457.36927509, ...,
         -6349581.07681948,  -3072808.2049037 ,  -3979152.47781669],
       [ 48216688.23799605,  -8666658.01688782,  21388001.6828212 , ...,
         49953950.22736988, -13569728.93089772,  15282197.55573998],
       ...,
       [-46611561.54191081,   -295671.43795167,  10516326.30378189, ...,
         -1339417.61911492,  10913521.77510528,   4838790.40477581],
       [-33783775.29316445,  13367791.86737388,   4226772.413175  , ...,
          8829791.57599485,  15498847.19026309, -10215953.21786739],
       [-26906873.37997736, -42350844.83762661, -13730224.72023512, ...,
         -2918177.5756186 ,   2037637.41883306,   -514914.51555043]])

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin

class drop_some_attributes(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X.drop(["회계년도","회사명"], axis=1)

class PCA_K2H(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        pca.fit(X[micro_val])
        return self
    def transform(self, X, y=None):
        self.index_ = X.index
        
        col = []
        for i in range(9):
            col.append('pca%s' %i)
        
        X9D = pd.DataFrame(pca.transform(X[micro_val]), 
                          index=self.index_, columns = col)
        
        X.drop(micro_val, axis=1, inplace=True)
        return X.join(X9D)
    
class date_of_establish(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X["설립일"] = X["설립일"].str.split('-', n = 1, expand=True)[0]
        return X

파이프라인 준비

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

cat_attributes = ["산업명", "소속코드", "기업규모명","설립일"]
num_attributes = []
for i in train.columns:
    if i not in cat_attributes:
        num_attributes.append(i)

In [19]:
num_pipeline = Pipeline([
        ('drop회사년도,회사명',drop_some_attributes()),
        ('PCA_micro_att',PCA_K2H()),
        ('StandardScaler',StandardScaler()),
    ])

In [20]:
cat_pipeline = Pipeline([
        ('설립일',date_of_establish()),
        ('onehotencoder',OneHotEncoder(categories='auto')),
    ])

In [21]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attributes),
        ("cat", cat_pipeline, cat_attributes),
    ])

In [22]:
train_prepared = full_pipeline.fit_transform(train)
train_prepared.shape

(15316, 144)

## 머신러닝 트레이닝

In [23]:
from sklearn.metrics import mean_squared_error

### LinearRegression

In [24]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(train_prepared, labels)

train_predictions = lin_reg.predict(train_prepared)
lin_mse = mean_squared_error(labels, train_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

17637.336321723575

### DecisionTreeRegressor

In [25]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state=42, max_depth=10)

tree_reg.fit(train_prepared, labels)

train_predictions = tree_reg.predict(train_prepared)
tree_mse = mean_squared_error(labels, train_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

8318.801689755004

### RandomForestRegressor

In [26]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=30, random_state=42, max_depth=50)
forest_reg.fit(train_prepared, labels)

train_predictions = forest_reg.predict(train_prepared)
forest_mse = mean_squared_error(labels, train_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

4864.16902739291

## cross_val_score

In [27]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print('Scores:', scores)
    print('Mean:', scores.mean())
    print('Standard deviation:', scores.std())

In [28]:
scores = cross_val_score(lin_reg, train_prepared, labels,
scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

Scores: [11148.39729631 16094.9442026  18321.29935333 15519.20745056
 18009.34374014 21351.43859114 14504.19878886 19403.22183731
 23775.4409792  17627.58032478]
Mean: 17575.507256422356
Standard deviation: 3375.583234666946


In [29]:
scores = cross_val_score(tree_reg, train_prepared, labels,
scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

Scores: [10245.35672567  9124.76838498 27911.43710175 13346.94350456
 20247.70660104 11032.28057673  9849.54896253 11572.81724435
 22786.56067992 21321.87776733]
Mean: 15743.929754886227
Standard deviation: 6349.638442095996


In [30]:
scores = cross_val_score(forest_reg, train_prepared, labels,
scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Scores: [ 8068.15526746  8325.20707376 18057.72046368  9303.48669195
 17277.76946136  8626.63650852  7414.11577374 11708.18588735
 18265.50553566  9071.91643272]
Mean: 11611.869909620087
Standard deviation: 4237.975601132525


### 그리드 탐색

In [31]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
    ]

forest_reg = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                        scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(train_prepared, labels)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [32]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [33]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=None, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [34]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

17481.992433041483 {'max_features': 2, 'n_estimators': 3}
15576.524890490266 {'max_features': 2, 'n_estimators': 10}
14664.19296948055 {'max_features': 2, 'n_estimators': 30}
17245.778455728778 {'max_features': 4, 'n_estimators': 3}
15470.978550536618 {'max_features': 4, 'n_estimators': 10}
14313.870839609357 {'max_features': 4, 'n_estimators': 30}
18935.89668386161 {'max_features': 6, 'n_estimators': 3}
14739.66586719307 {'max_features': 6, 'n_estimators': 10}
14174.98888253793 {'max_features': 6, 'n_estimators': 30}
17433.177811351205 {'max_features': 8, 'n_estimators': 3}
15328.218252252318 {'max_features': 8, 'n_estimators': 10}
14355.427411210085 {'max_features': 8, 'n_estimators': 30}
16910.356384607152 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
14773.895862641964 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
18809.01787218176 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
15300.829567993866 {'bootstrap': False, 'max_features': 3, 'n_esti

In [35]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=46, random_state=42, max_depth=45)
forest_reg.fit(train_prepared, labels)

train_predictions = forest_reg.predict(train_prepared)
forest_mse = mean_squared_error(labels, train_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

4503.191601011734

In [36]:
scores = cross_val_score(forest_reg, train_prepared, labels,
        scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Scores: [ 8126.40309802  8436.25508976 18052.47552142  9137.54178308
 17291.12287264  8970.88627758  7294.04561124 11564.69809008
 16996.84760568  9440.89117046]
Mean: 11531.116711996068
Standard deviation: 4016.9288268782766


### 랜덤 그리드 탐색

In [37]:
from sklearn.model_selection import RandomizedSearchCV

In [38]:
from scipy.stats import randint as sp_randint

In [39]:
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["mse"]}

n_iter_search = 20
random_search = RandomizedSearchCV(forest_reg, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5)
random_search.fit(train_prepared, labels)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=45,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=46, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=20, n_jobs=None,
          param_distributions={'max_depth': [3, None], 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000000182CFC18>, 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000000182CC940>, 'bootstrap': [True, False], 'criterion': ['mse']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [40]:
cvres = random_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(mean_score), params)

0.69139934047037 {'bootstrap': True, 'criterion': 'mse', 'max_depth': None, 'max_features': 4, 'min_samples_split': 5}
0.7676059479677879 {'bootstrap': False, 'criterion': 'mse', 'max_depth': None, 'max_features': 8, 'min_samples_split': 7}
0.7085746331342629 {'bootstrap': True, 'criterion': 'mse', 'max_depth': None, 'max_features': 5, 'min_samples_split': 7}
0.700773139069213 {'bootstrap': True, 'criterion': 'mse', 'max_depth': None, 'max_features': 8, 'min_samples_split': 8}
0.7120625187916523 {'bootstrap': True, 'criterion': 'mse', 'max_depth': None, 'max_features': 6, 'min_samples_split': 5}
0.2702706413554317 {'bootstrap': False, 'criterion': 'mse', 'max_depth': 3, 'max_features': 4, 'min_samples_split': 2}
0.26684204420119656 {'bootstrap': True, 'criterion': 'mse', 'max_depth': 3, 'max_features': 4, 'min_samples_split': 4}
0.38171257346123283 {'bootstrap': False, 'criterion': 'mse', 'max_depth': 3, 'max_features': 10, 'min_samples_split': 8}
0.7565406908448685 {'bootstrap': False

In [41]:
random_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=7, min_weight_fraction_leaf=0.0,
           n_estimators=46, n_jobs=None, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

#### 랜덤 포레스트 마지막 

In [42]:
labels.mean()

8890.279315748237

In [43]:
forest_reg = RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=45,
           max_features=9, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=3, min_weight_fraction_leaf=0.0,
           n_estimators=46, n_jobs=None, oob_score=False, random_state=42,
           verbose=0, warm_start=False)
forest_reg.fit(train_prepared, labels)

train_predictions = forest_reg.predict(train_prepared)
forest_mse = mean_squared_error(labels, train_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

1471.6147125584

In [44]:
scores = cross_val_score(forest_reg, train_prepared, labels,
        scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Scores: [ 8361.25335322  9018.7355717  12788.22630729  9948.97575157
 16022.36551421 10757.09421144  8447.33202661 12093.90445442
 19541.2558265  10693.97626021]
Mean: 11767.311927717608
Standard deviation: 3396.258572233329


## 딥러닝

In [57]:
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense
import random

def reset_graph(seed=42): ## 그래프에 변수가 쌓이는 것을 방지
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [58]:
train_prepared.shape

(15316, 144)

In [85]:
model = Sequential()
model.add(Dense(86, input_dim=144, activation='elu')) ## input * 2/3
model.add(Dense(86, activation='elu'))
model.add(Dense(40, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(1))

In [86]:
from keras import optimizers

In [87]:
sgd = optimizers.SGD(lr=0.0001, clipnorm=1.)

In [88]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [89]:
hist = model.fit(train_prepared, labels, epochs=500,batch_size=512)

Epoch 1/500
15316/15316 [==============================] - 0s 26us/step - loss: 507074463.1392
Epoch 2/500
15316/15316 [==============================] - 0s 8us/step - loss: 506476899.9237
Epoch 3/500
15316/15316 [==============================] - 0s 8us/step - loss: 502611505.7926
Epoch 4/500
15316/15316 [==============================] - 0s 8us/step - loss: 487787645.0541
Epoch 5/500
15316/15316 [==============================] - 0s 8us/step - loss: 449027377.5336
Epoch 6/500
15316/15316 [==============================] - 0s 8us/step - loss: 397206602.9564
Epoch 7/500
15316/15316 [==============================] - 0s 8us/step - loss: 374264559.6574
Epoch 8/500
15316/15316 [==============================] - 0s 8us/step - loss: 362012847.5362
Epoch 9/500
15316/15316 [==============================] - 0s 8us/step - loss: 351756319.0222
Epoch 10/500
15316/15316 [==============================] - 0s 8us/step - loss: 342658678.1635
Epoch 11/500
15316/15316 [==============================] 

15316/15316 [==============================] - 0s 9us/step - loss: 168327563.3387
Epoch 88/500
15316/15316 [==============================] - 0s 8us/step - loss: 167691339.0650
Epoch 89/500
15316/15316 [==============================] - 0s 8us/step - loss: 165548593.6756
Epoch 90/500
15316/15316 [==============================] - 0s 8us/step - loss: 163372312.7877
Epoch 91/500
15316/15316 [==============================] - 0s 8us/step - loss: 161702469.2505
Epoch 92/500
15316/15316 [==============================] - 0s 9us/step - loss: 160674771.7211
Epoch 93/500
15316/15316 [==============================] - 0s 8us/step - loss: 158701888.5516
Epoch 94/500
15316/15316 [==============================] - 0s 8us/step - loss: 158111896.4638
Epoch 95/500
15316/15316 [==============================] - 0s 8us/step - loss: 156148286.4247
Epoch 96/500
15316/15316 [==============================] - 0s 8us/step - loss: 154662190.4205
Epoch 97/500
15316/15316 [==============================] - 0s 

15316/15316 [==============================] - 0s 9us/step - loss: 64025967.1225
Epoch 173/500
15316/15316 [==============================] - 0s 8us/step - loss: 63948215.3481
Epoch 174/500
15316/15316 [==============================] - 0s 8us/step - loss: 62864161.1826
Epoch 175/500
15316/15316 [==============================] - 0s 9us/step - loss: 62166903.5362
Epoch 176/500
15316/15316 [==============================] - 0s 9us/step - loss: 63244195.2280
Epoch 177/500
15316/15316 [==============================] - 0s 8us/step - loss: 61532413.3967
Epoch 178/500
15316/15316 [==============================] - 0s 8us/step - loss: 62004354.0131
Epoch 179/500
15316/15316 [==============================] - 0s 8us/step - loss: 60420942.8707
Epoch 180/500
15316/15316 [==============================] - 0s 8us/step - loss: 60240042.4811
Epoch 181/500
15316/15316 [==============================] - 0s 8us/step - loss: 60371809.8417
Epoch 182/500
15316/15316 [==============================] - 0s 

15316/15316 [==============================] - 0s 8us/step - loss: 37008212.1295
Epoch 259/500
15316/15316 [==============================] - 0s 8us/step - loss: 36760651.2536
Epoch 260/500
15316/15316 [==============================] - 0s 8us/step - loss: 36458041.4432
Epoch 261/500
15316/15316 [==============================] - 0s 9us/step - loss: 36034521.4808
Epoch 262/500
15316/15316 [==============================] - 0s 8us/step - loss: 36391809.9044
Epoch 263/500
15316/15316 [==============================] - 0s 9us/step - loss: 36708604.0251
Epoch 264/500
15316/15316 [==============================] - 0s 8us/step - loss: 36740902.1259
Epoch 265/500
15316/15316 [==============================] - 0s 8us/step - loss: 36545612.0000
Epoch 266/500
15316/15316 [==============================] - 0s 9us/step - loss: 35881836.6968
Epoch 267/500
15316/15316 [==============================] - 0s 8us/step - loss: 35775668.2100
Epoch 268/500
15316/15316 [==============================] - 0s 

15316/15316 [==============================] - 0s 8us/step - loss: 27027781.3163
Epoch 345/500
15316/15316 [==============================] - 0s 8us/step - loss: 26885751.9013
Epoch 346/500
15316/15316 [==============================] - 0s 8us/step - loss: 27173000.1520
Epoch 347/500
15316/15316 [==============================] - 0s 9us/step - loss: 26856139.1011
Epoch 348/500
15316/15316 [==============================] - 0s 8us/step - loss: 26671743.1559
Epoch 349/500
15316/15316 [==============================] - 0s 9us/step - loss: 26803963.3351
Epoch 350/500
15316/15316 [==============================] - 0s 9us/step - loss: 26792839.9185
Epoch 351/500
15316/15316 [==============================] - 0s 9us/step - loss: 26710640.3181
Epoch 352/500
15316/15316 [==============================] - 0s 9us/step - loss: 26619829.2358
Epoch 353/500
15316/15316 [==============================] - 0s 9us/step - loss: 26707770.2173
Epoch 354/500
15316/15316 [==============================] - 0s 

15316/15316 [==============================] - 0s 8us/step - loss: 22189199.9846
Epoch 431/500
15316/15316 [==============================] - 0s 8us/step - loss: 21951513.8180
Epoch 432/500
15316/15316 [==============================] - 0s 8us/step - loss: 21626985.5069
Epoch 433/500
15316/15316 [==============================] - 0s 8us/step - loss: 21837335.5628
Epoch 434/500
15316/15316 [==============================] - 0s 8us/step - loss: 21710839.5511
Epoch 435/500
15316/15316 [==============================] - 0s 8us/step - loss: 22363215.3220
Epoch 436/500
15316/15316 [==============================] - 0s 8us/step - loss: 21622023.6819
Epoch 437/500
15316/15316 [==============================] - 0s 8us/step - loss: 21441101.2823
Epoch 438/500
15316/15316 [==============================] - 0s 8us/step - loss: 21274060.1995
Epoch 439/500
15316/15316 [==============================] - 0s 8us/step - loss: 21480760.3382
Epoch 440/500
15316/15316 [==============================] - 0s 